# # Project: NLP to Analyze Topics of Consumer Reviews of AmazonAutomatives   

In this project, we will use Topic Modeling to understand consumer reviews. Our aim here is to extract a number of high-level topics each consists og important words with the similar theme from the reviews. These groups of words are basically the topics which would help in ascertaining what the consumers are actually talking about in the reviews.

We specifically use Latent Dirichlet Allocation (LDA) to extract useful topics from our online reviews dataset. 
The dataset is Amazon Automotive Review.

In [70]:
import pandas as pd
import nltk
from nltk import FreqDist
# nltk.download('stopwords')

## Part 1: Load and Select the Data 

In [71]:
reviews = pd.read_json('11 ML_NLPTopicModeling_KaggleChallengeHotelReviewDataset.json', lines=True)
reviews.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


In [72]:
# Select only the relevant data for Sentiment Analysis
reviews_SA = reviews[["reviewText"]]
reviews_SA.head()

,reviewText
0,I needed a set of jumper cables for my new car...
1,"These long cables work fine for my truck, but ..."
2,Can't comment much on these since they have no...
3,I absolutley love Amazon!!! For the price of ...
4,I purchased the 12' feet long cable set and th...


## Part 2: Text Preprocessing 

## Part 2A: Lower the Text 

In [73]:
# lower text
def text_low(text):
    text = text.lower()
    return text 
reviews_SA["Review_clean"] = reviews['reviewText'].apply(lambda x: text_low(x))
reviews_SA.head(10)

/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,i needed a set of jumper cables for my new car...
1,"These long cables work fine for my truck, but ...","these long cables work fine for my truck, but ..."
2,Can't comment much on these since they have no...,can't comment much on these since they have no...
3,I absolutley love Amazon!!! For the price of ...,i absolutley love amazon!!! for the price of ...
4,I purchased the 12' feet long cable set and th...,i purchased the 12' feet long cable set and th...
5,"These Jumper cables are heavy Duty, Yet easy t...","these jumper cables are heavy duty, yet easy t..."
6,bought these for my k2500 suburban plenty of l...,bought these for my k2500 suburban plenty of l...
7,these are good enough to get most motorized ve...,these are good enough to get most motorized ve...
8,The Coleman Cable 08665 12-Feet Heavy-Duty Tru...,the coleman cable 08665 12-feet heavy-duty tru...
9,"I have an old car, Its bound to need these som...","i have an old car, its bound to need these som..."


## Part 2B: Tokenize 

In [74]:
# tokenize text and remove puncutation
from nltk.tokenize import WhitespaceTokenizer
import string
def text_tok(text):
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    return text

reviews_SA["Review_clean"] = reviews_SA["Review_clean"].apply(lambda x: text_tok(x))
reviews_SA.head(3)


/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,"[i, needed, a, set, of, jumper, cables, for, m..."
1,"These long cables work fine for my truck, but ...","[these, long, cables, work, fine, for, my, tru..."
2,Can't comment much on these since they have no...,"[can't, comment, much, on, these, since, they,..."


## Part 2C: Noise Removal 

In [75]:
from nltk.corpus import stopwords
def text_noise(text):
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove useless stop words like 'the', 'a' ,'this' etc.
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    return text 

reviews_SA["Review_clean"] = reviews_SA["Review_clean"].apply(lambda x: text_noise(x))
reviews_SA.head(3)

/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,"[needed, set, jumper, cables, new, car, good, ..."
1,"These long cables work fine for my truck, but ...","[long, cables, work, fine, truck, quality, see..."
2,Can't comment much on these since they have no...,"[can't, comment, much, since, yet, used, come,..."


## Part 2D: POS Tagging 

In [76]:
from nltk import pos_tag
def text_pos(text):
    text = pos_tag(text)
    return text
reviews_SA["Review_clean"] = reviews_SA["Review_clean"].apply(lambda x: text_pos(x))
reviews_SA.head(3)

/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,"[(needed, VBN), (set, VBN), (jumper, NN), (cab..."
1,"These long cables work fine for my truck, but ...","[(long, JJ), (cables, NNS), (work, VBP), (fine..."
2,Can't comment much on these since they have no...,"[(can't, JJ), (comment, NN), (much, RB), (sinc..."


## Part 2E: Lemmatize 

In [77]:
# WordNet® is a large lexical database of English.
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# We need to map NLTK’s POS tags to the format wordnet lemmatizer would accept. 
# The get_wordnet_pos() function defined below does this mapping job.
def get_wordnet_pos(t):
    if t.startswith('J'):
        return wordnet.ADJ
    elif t.startswith('V'):
        return wordnet.VERB
    elif t.startswith('N'):
        return wordnet.NOUN
    elif t.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# lemmatize text
# we use POS tag as the second argument to lemmatize() to make sure it gives us the right root.
def text_lem(text):
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in text]
    return text

reviews_SA["Review_clean"] = reviews_SA["Review_clean"].apply(lambda x: text_lem(x))
reviews_SA.head(3)

/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,"[need, set, jumper, cable, new, car, good, rev..."
1,"These long cables work fine for my truck, but ...","[long, cable, work, fine, truck, quality, seem..."
2,Can't comment much on these since they have no...,"[can't, comment, much, since, yet, use, come, ..."


## Part 2F: Final Clean-up

In [88]:
def text_cl(text):
    # remove words with only one letter
    text = [word for word in text if len(word) > 2]
    # join all
    # text = " ".join(text)
    return text

reviews_SA["Review_clean"] = reviews_SA["Review_clean"].apply(lambda x: text_cl(x))
reviews_SA.head(3)

/anaconda3/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,reviewText,Review_clean
0,I needed a set of jumper cables for my new car...,"[need, set, jumper, cable, new, car, good, rev..."
1,"These long cables work fine for my truck, but ...","[long, cable, work, fine, truck, quality, seem..."
2,Can't comment much on these since they have no...,"[can't, comment, much, since, yet, use, come, ..."


## Part 3: Topic Modeling

Topic Modelling is different from rule-based text mining approaches that use regular expressions or dictionary based keyword searching techniques. It is an unsupervised approach used for finding and observing the bunch of words (called “topics”) in large clusters of texts.

## Part 3A: Frequency Calculation Function

In [91]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

    
def freq_words(text, terms = 30):
  all_words = ' '.join([x for x in text])
  all_words = all_words.split()
  
  fdist = FreqDist(all_words)
  words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
  
  # selecting top 20 most frequent words
  d = words_df.nlargest(columns="count", n = terms) 
  plt.figure(figsize=(20,5))
  ax = sns.barplot(data=d, x= "word", y = "count")
  ax.set(ylabel = 'Count')
  plt.show()

In [92]:
freq_words(reviews_SA['Review_clean'])

TypeError: sequence item 0: expected string, list found

## Part 3A: Building an LDA model

LDA assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. Given a dataset of documents, LDA backtracks and tries to figure out what topics would create those documents in the first place.

Good ref on topic modeling and LDA:
https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

In [94]:
import gensim
from gensim import corpora
# creating the term dictionary of our corpus
dictionary = corpora.Dictionary(reviews_SA['Review_clean'])
# convert the list of reviews into a Document Term Matrix using the dictionary prepared above
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_SA['Review_clean']]

In [95]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model with 7 topics 
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,chunksize=1000, passes=50)

In [96]:
lda_model.print_topics()

[(0,
  u'0.019*"jack" + 0.012*"cut" + 0.010*"stand" + 0.010*"hitch" + 0.009*"lock" + 0.009*"grease" + 0.007*"rust" + 0.007*"key" + 0.007*"board" + 0.007*"surface"'),
 (1,
  u'0.019*"use" + 0.017*"one" + 0.016*"get" + 0.015*"work" + 0.012*"well" + 0.012*"good" + 0.011*"time" + 0.010*"car" + 0.010*"buy" + 0.010*"would"'),
 (2,
  u'0.047*"oil" + 0.036*"filter" + 0.020*"change" + 0.016*"engine" + 0.012*"fuel" + 0.011*"fluid" + 0.009*"use" + 0.008*"drain" + 0.008*"mile" + 0.007*"run"'),
 (3,
  u'0.035*"light" + 0.026*"battery" + 0.015*"power" + 0.013*"charge" + 0.013*"plug" + 0.012*"bulb" + 0.011*"use" + 0.011*"car" + 0.010*"device" + 0.009*"bright"'),
 (4,
  u'0.026*"use" + 0.022*"car" + 0.015*"product" + 0.015*"clean" + 0.012*"towel" + 0.010*"blade" + 0.010*"wash" + 0.010*"well" + 0.010*"like" + 0.009*"wiper"'),
 (5,
  u'0.019*"tire" + 0.016*"hose" + 0.011*"use" + 0.010*"tool" + 0.010*"handle" + 0.009*"pressure" + 0.009*"get" + 0.009*"make" + 0.009*"gauge" + 0.008*"one"'),
 (6,
  u'0.019*

In [99]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

/anaconda3/anaconda3/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      32.299297        1       1 -0.202826  0.055252
4      24.968403        1       2 -0.154252  0.048773
3      12.644479        1       3  0.008006  0.077318
5      10.937506        1       4 -0.077117 -0.078751
6      10.462372        1       5 -0.137674 -0.086480
2       4.588692        1       6  0.291374  0.239435
0       4.099257        1       7  0.272488 -0.255547, topic_info=      Category         Freq       Term         Total  loglift  logprob
term                                                                  
28975  Default  4469.000000      light   4469.000000  30.0000  30.0000
13863  Default  2805.000000    battery   2805.000000  29.0000  29.0000
25604  Default  1821.000000        oil   1821.000000  28.0000  28.0000
7520   Default  1420.000000     filter   1420.000000  27.0000  27.0000
33405  Default  3352.000000      clean   3352.000000  26.0000  26.0000
23710  Default  4252.000000       look   4252.000000  25.0000  25.0000
24246  Default  2105.000000       tire   2105.000000  24.0000  24.0000
22786  Default  8595.000000        car   8595.000000  23.0000  23.0000
31382  Default  5018.000000      great   5018.000000  22.0000  22.0000
8401   Default  2604.000000      towel   2604.000000  21.0000  21.0000
155    Default  2718.000000        fit   2718.000000  20.0000  20.0000
12907  Default  1377.000000     change   1377.000000  19.0000  19.0000
30566  Default  1567.000000      power   1567.000000  18.0000  18.0000
36710  Default  1527.000000       hose   1527.000000  17.0000  17.0000
9899   Default  2227.000000      blade   2227.000000  16.0000  16.0000
10042  Default  2203.000000       wash   2203.000000  15.0000  15.0000
27605  Default  1439.000000     charge   1439.000000  14.0000  14.0000
29914  Default  3880.000000       easy   3880.000000  13.0000  13.0000
27499  Default  1411.000000       plug   1411.000000  12.0000  12.0000
25867  Default  2004.000000      wiper   2004.000000  11.0000  11.0000
22629  Default  1940.000000        wax   1940.000000  10.0000  10.0000
24602  Default  1929.000000        dry   1929.000000   9.0000   9.0000
5540   Default  1389.000000    install   1389.000000   8.0000   8.0000
25003  Default  5550.000000       make   5550.000000   7.0000   7.0000
26769  Default  1253.000000       bulb   1253.000000   6.0000   6.0000
28012  Default  1049.000000     engine   1049.000000   5.0000   5.0000
24388  Default  6811.000000       well   6811.000000   4.0000   4.0000
3163   Default  6085.000000    product   6085.000000   3.0000   3.0000
26987  Default   660.000000       jack    660.000000   2.0000   2.0000
4671   Default  1143.000000    surface   1143.000000   1.0000   1.0000
...        ...          ...        ...           ...      ...      ...
24958   Topic7    84.155098     height     84.877838   3.1858  -6.0292
5234    Topic7    84.210022  resistant     84.938187   3.1858  -6.0286
24384   Topic7    81.371605       weld     82.093277   3.1855  -6.0629
19603   Topic7    77.541183     saddle     78.266304   3.1851  -6.1111
14359   Topic7    77.170547        bed     77.893982   3.1850  -6.1159
595     Topic7    76.287262      chain     77.009644   3.1849  -6.1274
21243   Topic7    71.611259      bulky     72.339294   3.1842  -6.1907
5976    Topic7    69.451035      knife     70.175507   3.1840  -6.2213
7189    Topic7    69.496651       hair     70.224045   3.1840  -6.2206
33567   Topic7    68.341377   radiator     69.066620   3.1838  -6.2374
26452   Topic7    68.441704    account     69.177063   3.1837  -6.2359
16540   Topic7    63.674137     groove     64.399292   3.1830  -6.3081
14908   Topic7   302.369263     grease    355.956451   3.0312  -4.7503
11890   Topic7   184.431778      steel    210.018311   3.0644  -5.2446
8710    Topic7   234.235489     weight    345.047882   2.8070  -5.0056
27988   Topic7   243.202362        key    378.188568   2.7529  